# BEER TOF diagram

This notebook serves to check the TOF diagrams of the BEER instrument chopper cascade.  
The operation modes are identical to the ones defined in <a href='https://chess.esss.lu.se/enovia/link/ESS-0238217/21308.51166.9984.44949/valid'>ESS-0238217</a> - *Neutron Optics Report for the BEER Instrument*. 

The definition of the chopper modes, the plotting and calculating functions are located in a separate Python file, which is imported below (`beer_choppers_tof`). The predefined chopper mode can be listed and changed, or you can create a new one. The position of the monitor can also be modified.

## Basic description
### Python packages

This Jupyter notebook uses the following packages for plotting and interactions: `matplotlib` and `ipywidgets`.

### BEER choppers TOF package

In the `beer_choppers_tof` package, there is a dependency on `scipp`, `scippneutron`, `plopp` and `tof` packages, which can be installed via `pip install scippneutron tof`. The necessary version of the `tof` package to work with is >= **25.5.0**. 

*Note:* If you use **conda**, use the appropriate command for installing the package into that environment.

When the package is imported, it defines and fills up some general variables which can be used:<br>
`class beer_mode` - a structure which holds the defined chopper setting for the operation mode (see below)<br>
`modes` - list of all predefined chopper modes <br>
`detectors` - predefined detectors setting <br>
`lambda_0` - mean wavelength <br>

**Definition of the units:**<br>
`Hz` - Hertz<br>
`deg` - degree <br>
`m` - meter <br>
`A` - Angstrom <br>
`s` - seconds<br>
Don't use the variables with the same names.

### Useful functions
`load_modes` - recreates/resets `modes` with the predefined ones <br>
*Note: this function is called automatically during the import of the package `beer_choppers_tof`*<br>
`print_modes_info`, `print_choppers_info`, `print_detectors_info` - print info about available items<br>
`draw_chopper` - draws graphical representation of the chopper<br>
`run_and_plot_mode` - runs the simulation and plots the TOF diagram <br>
`plot_choppers` - provides output from `run_and_plot_mode` and plots the choppers info<br>
`plot_detectors` - provides output from `run_and_plot_mode` and plots the detector info<br>
`get_wave_for_all_modes` - calculates the TOF from all the modes and stores the information of the wavelength distribution for the future comparison <br>
`plot_comparison` - takes the results from `get_wave_for_all_modes` and plots several modes and/or pulses together
`plot_ess_pulse` - plots the ESS pulse structure <br>

*Note:* All the indexes are zero-based! 

## Loading of the basic packages

In [ ]:
# ruff: noqa
from ipywidgets import widgets, interactive
import matplotlib.pyplot as plt
import numpy as np
import ess.beer.choppers as beer

%matplotlib widget

## The ESS pulse
Visualisation of the ESS pulse structure.

In [ ]:
save_ess_button = widgets.Button(description='Save to PDF')
data_ess_button = widgets.Button(description='Save data')
output_ess = widgets.Output()

# display the selection, buttons and output
display(widgets.HBox([save_ess_button, data_ess_button]), output_ess)

with output_ess:
    ess_pulse = beer.plot_ess_pulse()

def save_ess_on_click(b):
    """ Save plot to PDF """ 
    with output_ess:
        ess_pulse.savefig('ESS_source_pulse.pdf', format='pdf')
        print('Figure saved to PDF.')
        
def data_ess_on_click(b):
    """ Save plotted data to ASCII file """
    with output_ess:
        for ax in ess_pulse.axes:
            xl = ax.xaxis.get_label().get_text().split(' ')[0]
            yl = ax.yaxis.get_label().get_text().split(':')[0]
            a = np.array(ax.lines[0].get_xydata())
            fl = f'ESS-pulse-{xl}_{yl}.dat'
            np.savetxt(fl, a, delimiter=" ")
            print(f'Data saved to "{fl}".')

save_ess_button.on_click(save_ess_on_click)
data_ess_button.on_click(data_ess_on_click)

## Pre-loaded modes
Summary information of all standard pre-loaded modes of the BEER instrument.  
You can list all the modes when not providing any parameter to the function or put ```index``` or ```mode_id``` string to get specific mode information.

*Examples:*

```beer.print_modes_info('M2')```

```beer.print_modes_info('4')```

```beer.print_modes_info('all')```

In [ ]:
beer.print_modes_info('all')

## Chopper visualisation
`beer.print_choppers_info` prints a list of all choppers in the selected mode, and `beer.draw_chopper` shows the graphic representation of the chopper disk. The choppers are drawn in the time t$_0$ configuration.<br>

*Usage:* Change the indexes as necessary. If you want to draw more choppers, copy and adapt the code in the next cell. 

*Examples:*  
```beer.print_choppers_info(5)```  
```beer.draw_chopper(5, 1)```

In [ ]:
mode_index = beer.get_mode_index('PS2') # getting mode index by string
chopper_index = beer.get_chopper_index(mode_index, 'PSC2') # getting chopper index by name

beer.print_choppers_info(mode_index) # list all choppers in the mode
beer.draw_chopper(mode_index, chopper_index) # draw the chopper

## Show TOF diagram

### Mode updates
In BEER basic setup, the **FC1A** chopper has a frequency of 28 Hz. This setting allows penetration of the neutrons with a wavelength above 15 Å to go through the chopper cascade, each 2nd pulse via **FC1A** and each 6th pulse via **FC2A**.  

The intensity of those long-wavelength neutrons is very small. You can omit it by setting ```wave_max``` to 15 Å.  
To fully suppress the overlap, **FC1A** could be run with a frequency of 14 Hz. The code to change the setting of the frequency is implemented below and can be uncommented when necessary. One can also play with the chopper additional shift if necessary.<br>

*Note:* Be aware that the change in the frequency of the mode affects the other results visualisation. To reload predefined modes use `beer.load_modes()` function.

You can skip the following cell if you want to continue with the original modes.

In [ ]:
beer.load_modes()
##### Code to change the frequency of the FC1A chopper ######
# For the Pulse shaping modes affected, the frequency of the FC1A chopper 
# has to be changed to 14 Hz from 28 Hz, no shift of the offset is needed
new_fc1a_freq = 14.

modes = ['PS1', 'PS2', 'PS3']
choppers = ['FC1A']

for mode in modes:
    m_index = beer.get_mode_index(mode)
    for chopper in choppers:
        ch_index = beer.get_chopper_index(m_index, chopper)
        beer.modes[m_index].choppers[ch_index].frequency = new_fc1a_freq * beer.Hz # change the frequency

# For the Modulation modes affected, the frequency of the FC1A chopper 
# has to be changed to 14 Hz, and the shift of the offset to 15º from the original 6º
modes = ['8X - M0', '8X - M2', '8X - M3', '16X - M0', '16X - M2', '16X - M3']
choppers = ['FC1A']

for mode in modes:
    m_index = beer.get_mode_index(mode)
    for chopper in choppers:
        ch_index = beer.get_chopper_index(m_index, chopper)
        beer.modes[m_index].choppers[ch_index].frequency = new_fc1a_freq * beer.Hz # change the frequency
        off = 15*beer.deg + beer.modes[m_index].choppers[ch_index].close[0]/2
        beer.modes[m_index].chopper_offset[ch_index] = off # change the offset

# recalculate the chopper phase shifts
for mode in beer.modes:
    mode.update_chopper_phases()
##############################################################

###### Adding some testing modes ######
###### First additional mode
fMC = 84.
mode = beer.beer_mode(2.1 * beer.A)
mode.caption = 'modulation (HF) 4X - M0+M1'
#                distance  freq.     offset        title   opening/closing
mode.add_chopper(8.283*beer.m, new_fc1a_freq*beer.Hz, (72/2+15)*beer.deg, 'FC1A', [0], [72])
mode.add_chopper(9.300*beer.m, (fMC)*beer.Hz, 2.5*beer.deg,  'MCA',
                 list(on for on in np.arange(0.0, 360, 45)),
                 list(on for on in np.arange(5.0, 360, 45)))
mode.add_chopper(9.350*beer.m, (fMC/4)*beer.Hz, 2.5*beer.deg,  'MCB',
                 list(on for on in np.arange(0.0, 360, 22.5)),
                 list(on for on in np.arange(5.0, 360, 22.5)))
mode.add_chopper(79.975*beer.m, 14*beer.Hz, 175/2*beer.deg, 'FC2A', [0], [175])
modes.append(mode)

beer.modes.append(mode)

###### Second additional mode
fMC = 140.
mode = beer.beer_mode(2.1 * beer.A)
mode.caption = 'modulation (MR) 4X - M2'
#                distance  freq.     offset        title   opening/closing
mode.add_chopper(8.283*beer.m, new_fc1a_freq*beer.Hz, (72/2+15)*beer.deg, 'FC1A', [0], [72])
mode.add_chopper(9.300*beer.m, fMC*beer.Hz, 2.5*beer.deg,  'MCA',
                 list(on for on in np.arange(0.0, 360, 45)),
                 list(on for on in np.arange(5.0, 360, 45)))
mode.add_chopper(9.350*beer.m, (fMC/4)*beer.Hz, 2.5*beer.deg,  'MCB',
                 list(on for on in np.arange(0.0, 360, 22.5)),
                 list(on for on in np.arange(5.0, 360, 22.5)))
mode.add_chopper(79.975*beer.m, 14*beer.Hz, 175/2*beer.deg, 'FC2A', [0], [175])
modes.append(mode)

beer.modes.append(mode)

###### Third additional mode
fMC = 280.
mode = beer.beer_mode(2.1 * beer.A)
mode.caption = 'modulation (HR) 4X - M3'
#                distance  freq.     offset        title   opening/closing
mode.add_chopper(8.283*beer.m, new_fc1a_freq*beer.Hz, (72/2+15)*beer.deg, 'FC1A', [0], [72])
mode.add_chopper(9.300*beer.m, fMC*beer.Hz, 2.5*beer.deg,  'MCA',
                 list(on for on in np.arange(0.0, 360, 45)),
                 list(on for on in np.arange(5.0, 360, 45)))
mode.add_chopper( 9.350*beer.m, (fMC/4)*beer.Hz, 2.5*beer.deg,  'MCB',
                 list(on for on in np.arange(0.0, 360, 22.5)),
                 list(on for on in np.arange(5.0, 360, 22.5)))
mode.add_chopper(79.975*beer.m,  14.*beer.Hz, 175/2*beer.deg, 'FC2A', [0], [175])
modes.append(mode)

beer.modes.append(mode)

###### Fourth additional mode
fMC = 280.
mode = beer.beer_mode(2.1 * beer.A)
mode.caption = 'modulation (HR) 4X (JS) - M3'
#                distance  freq.     offset        title   opening/closing
mode.add_chopper( 8.283*beer.m, new_fc1a_freq*beer.Hz, (72/2+15)*beer.deg, 'FC1A', [0], [72])
mode.add_chopper( 9.300*beer.m, (fMC/2)*beer.Hz,   2.5*beer.deg,  'MCA',
                 list(on for on in np.arange(0.0, 360, 45)),
                 list(on for on in np.arange(5.0, 360, 45)))
mode.add_chopper( 9.350*beer.m, fMC*beer.Hz,       2.5*beer.deg,  'MCB',
                 list(on for on in np.arange(0.0, 360, 22.5)),
                 list(on for on in np.arange(5.0, 360, 22.5)))
mode.add_chopper(79.975*beer.m,  14*beer.Hz,     175/2*beer.deg, 'FC2A', [0], [175])
modes.append(mode)

beer.modes.append(mode)

###### Fifth additional mode
mode = beer.beer_mode(2.1 * beer.A)
mode.caption = f'modulation 4X ({fMC} Hz - double frame)'
#                distance  freq.     offset        title   opening/closing
mode.add_chopper( 8.283*beer.m,   7*beer.Hz, (72/2+12)*beer.deg, 'FC1A', [0], [72])
mode.add_chopper( 9.300*beer.m, fMC*beer.Hz,       2.5*beer.deg,  'MCA',
                 list(on for on in np.arange(0.0, 360, 45)),
                 list(on for on in np.arange(5.0, 360, 45)))
mode.add_chopper( 9.350*beer.m, (fMC/4)*beer.Hz,   2.5*beer.deg,  'MCB',
                 list(on for on in np.arange(0.0, 360, 22.5)),
                 list(on for on in np.arange(5.0, 360, 22.5)))
mode.add_chopper(79.975*beer.m,   7*beer.Hz,     175/2*beer.deg, 'FC2A', [0], [175])
modes.append(mode)

beer.modes.append(mode)

In [ ]:
# beer.print_modes_info('all')

### TOF plotting
The defined modes and number of simulated pulses can be selected.<br>
You can adjust the maximum wavelength (```wave_max```) in the simulation. ESS source provides neutrons up to $\lambda_{max} = 20$ Å. The default value is 15 Å.<br>

*Note:* You can increase the amount of neutrons to simulate when the statistic is necessary. 

In [ ]:
index, pulses = 8, 2  # start with PS2 and two pulses
wave_max = 20  # change if necessary - maximum 20

select_mode = widgets.Dropdown(options=list(f'{mode.caption}' for mode in beer.modes),
                               value=f'{beer.modes[index].caption}', 
                               description='Mode: ')
select_pulses = widgets.IntSlider(min=1, max=6, value=pulses, step=1, description='Pulses:')
recalculate_button = widgets.Button(description='Recalculate', disabled=True)
save_button = widgets.Button(description='Save to PDF', disabled=True)
output_tof = widgets.Output()     

# display the selection, buttons and output
display(widgets.HBox([select_mode, select_pulses, recalculate_button, save_button]), output_tof)

with output_tof:
    res = beer.run_and_plot_mode('', pulses, wmax=wave_max, mode_index=index, neutrons=1_000_000)
    save_button.disabled = False

# definition of the events
def select_mode_on_change(change):
    global index  
    index = change.new
    recalculate_button.disabled = False
    save_button.disabled = True
    output_tof.clear_output()
    with output_tof:
        plt.close(res[2])

def pulses_on_change(change):
    global pulses
    pulses = change.new
    recalculate_button.disabled = False
    save_button.disabled = True
    output_tof.clear_output()
    with output_tof:
        plt.close(res[2])

def recalculate_on_click(b):
    global res
    output_tof.clear_output()
    with output_tof:
        res = beer.run_and_plot_mode('', pulses, wmax=wave_max, mode_index=index, neutrons=1_000_000)
    is_ready = True
    recalculate_button.disabled = True
    save_button.disabled = False

def save_on_click(b):
    with output_tof:
        res[2].savefig(f'BEER-TOF-{beer.modes[index].caption}.pdf', format='pdf')
        print('Figure saved to PDF.')
    
select_mode.observe(select_mode_on_change, names='index')
select_pulses.observe(pulses_on_change, names='value')
recalculate_button.on_click(recalculate_on_click)
save_button.on_click(save_on_click)

## Analysis of the above simulated TOF

### Detector information
Select the available detector and the number of pulses to plot.

In [ ]:
d_index = -1 # print the last detector (sample position) as default
select_det = widgets.Dropdown(options=list(f'{det.name}' for det in beer.detectors),
                               value=f'{beer.detectors[d_index].name}', description='Detector: ')
save_det_button = widgets.Button(description='Save to PDF')
dat_det_button = widgets.Button(description='Save data')
d_pulses = []
det_name = beer.detectors[0].name
da_toa = res[1].detectors[det_name].data.sum('event').values

for i in range(pulses):
    # evaluating if the pulse is empty or not and disabling the checkbox
    enable = da_toa[i] > 0
    
    select = widgets.Checkbox(value=True if i == 0 else False, description=f'Pulse:{i+1}',
                              disabled=False if enable != 0 else True)
    d_pulses.append(select)

output_det = widgets.Output()

# display the selection, buttons and output
display(widgets.HBox([select_det, save_det_button, dat_det_button]), 
        widgets.HBox([cb for cb in d_pulses]), output_det)

def get_d_pulse_indexes():
    out = []
    for i, check in enumerate(d_pulses):
        if check.value: out.append(i)           
    return out

with output_det:
    fig_det = beer.plot_detectors(res[1], res[0], d_index, get_d_pulse_indexes())

# definition of the events
def select_det_on_change(change):
    global d_index, fig_det
    d_index = change.new
    output_det.clear_output()
    with output_det:
        plt.close(fig_det)
        fig_det = beer.plot_detectors(res[1], res[0], d_index, get_d_pulse_indexes())

def save_det_on_click(b):
    with output_det:
        fig_det.savefig(f'{beer.detectors[d_index].name} - {beer.modes[res[0]].caption}.pdf', format='pdf')
        print('Figure saved to PDF.')

def dat_det_on_click(b):
    with output_det:
        for ax in fig_det.axes:
            xl = ax.xaxis.get_label().get_text().split(' ')[0]
            yl = ax.yaxis.get_label().get_text().split(' ')[0]
            for line in ax.lines:
                a = np.array(line.get_xydata())
                fl = f'{beer.detectors[d_index].name}-{beer.modes[res[0]].caption}'
                
                fl = f"{fl}-{line.get_label().replace(': ', '_')}_{xl}_{yl}.dat"
                np.savetxt(fl, a, delimiter=" ")
                print(f'Data saved to "{fl}".')
        
def d_pulses_on_change(change):
    global fig_det
    output_det.clear_output()
    with output_det:
        plt.close(fig_det)
        pulse = get_d_pulse_indexes()
        if len(pulse) == 0:
            d_pulses[0].value = True
        else:
            fig_det = beer.plot_detectors(res[1], res[0], d_index, pulse)
    
select_det.observe(select_det_on_change, names='index')
save_det_button.on_click(save_det_on_click)
dat_det_button.on_click(dat_det_on_click)
for check in d_pulses:
    check.observe(d_pulses_on_change, names='value')

### Chopper information
Select the available chopper and the number of pulses to plot. 

In [ ]:
chop_index = 0
select_chop = widgets.Dropdown(options=list(f'{chop.name}' for chop in beer.modes[index].choppers),
                               value=f'{beer.modes[index].choppers[chop_index].name}', 
                               description='Chopper: ')
save_chop_button = widgets.Button(description='Save to PDF')
dat_chop_button = widgets.Button(description='Save data')
c_pulses = []
det_name = beer.detectors[0].name
da_toa = res[1].detectors[det_name].data.sum('event').values

for i in range(pulses):
    # evaluating if the pulse is empty or not and disabling the checkbox
    enable = da_toa[i] > 0

    select = widgets.Checkbox(value=True if i == 0 else False, 
                              description=f'Pulse:{i+1}',
                              disabled=False if enable != 0 else True)
    c_pulses.append(select)

output_chop = widgets.Output()

# display the selection, buttons and output
display(widgets.HBox([select_chop, save_chop_button, dat_chop_button]), 
        widgets.HBox([cb for cb in c_pulses]), output_chop)

def get_c_pulse_indexes():
    out = []
    for i, check in enumerate(c_pulses):
        if check.value: out.append(i)            
    return out

with output_chop:
    fig_chop = beer.plot_choppers(res[1], res[0], chop_index, get_c_pulse_indexes())

# definition of the events
def select_chop_on_change(change):
    global chop_index, fig_chop
    chop_index = change.new
    output_chop.clear_output()

    with output_chop:
        plt.close(fig_chop)
        fig_chop = beer.plot_choppers(res[1], res[0], chop_index, get_c_pulse_indexes())

def save_chop_on_click(b):
    with output_chop:
        fig_chop.savefig(f'{beer.modes[res[0]].choppers[chop_index].name}-{beer.modes[res[0]].caption}.pdf',
                         format='pdf')
        print('Figure saved to PDF.')

def dat_chop_on_click(b):
    with output_chop:
        for ax in fig_chop.axes:
            xl = ax.xaxis.get_label().get_text().split(' ')[0]
            yl = ax.yaxis.get_label().get_text().split(' ')[0]
            for line in ax.lines:
                a = np.array(line.get_xydata())
                fl = f'{beer.modes[res[0]].choppers[chop_index].name}-{beer.modes[res[0]].caption}'
                fl = f"{fl}-{line.get_label().replace(': ', '_')}_{xl}_{yl}.dat"
                np.savetxt(fl, a, delimiter=" ")
                print(f'Data saved to "{fl}".')
        
def c_pulses_on_change(change):
    global fig_chop
    output_chop.clear_output()
    with output_chop:
        plt.close(fig_chop)
        pulse = get_c_pulse_indexes()
        if len(pulse) == 0:
            check_pulses[0].value = True
        else:
            fig_chop = beer.plot_choppers(res[1], res[0], chop_index, pulse)
    
select_chop.observe(select_chop_on_change, names='index')
save_chop_button.on_click(save_chop_on_click)
dat_chop_button.on_click(dat_chop_on_click)
for check in c_pulses:
    check.observe(c_pulses_on_change, names='value')

## Comparison between various modes on the wavelength level

### Simulate all the modes
Variable ```waves``` store the wavelength results for all run modes. 

In [ ]:
waves = beer.get_wave_for_all_modes(neutrons=500_000, pulses=2, wmax=20.0)

### Plot the comparison
Below are some predefined comparisons. 

In [ ]:
beer.plot_comparison(waves, ['DS0', 'DS1'], [[0], [0, 1]], save_fig=False)

In [ ]:
beer.plot_comparison(waves, ['PS1', 'PS2', 'PS3'], save_fig=False)

In [ ]:
beer.plot_comparison(waves, ['8X - M0', '8X - M2', '8X - M3'], save_fig=False)

In [ ]:
beer.plot_comparison(waves, ['16X - M0', '16X - M2', '16X - M3'], save_fig=False)

In [ ]:
beer.plot_comparison(waves, ['8X - M0', '16X - M0', 'PS1', 'PS2'], save_fig=False)

In [ ]:
beer.plot_comparison(waves, ['4X - M2', '8X - M0', '8X - M2', '16X - M2', 'PS2'], save_fig=False)

**end of notebook**